In [23]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation


import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff


# Step 1: Loading Dataset

In [24]:
# Dataset Source: Kaggle
# Link: https://www.kaggle.com/datasets/sanchitagholap/crop-and-fertilizer-dataset-for-westernmaharashtra 

In [25]:
df = pd.read_csv('Data/Crop and fertilizer dataset.csv')

# Removing leading and trailing spaces from column names
df.columns = df.columns.str.strip()

# Replace multiple spaces with a single space in all column names
df.columns = df.columns.str.replace(r'\s+', ' ', regex=True)  

# Drop Unnecessary Columns
df.drop(['District_Name', 'Link'], axis=1, inplace=True)

# Step 2: Overview of Dataset

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4513 entries, 0 to 4512
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Soil_color   4513 non-null   object 
 1   Nitrogen     4513 non-null   int64  
 2   Phosphorus   4513 non-null   int64  
 3   Potassium    4513 non-null   int64  
 4   pH           4513 non-null   float64
 5   Rainfall     4513 non-null   int64  
 6   Temperature  4513 non-null   int64  
 7   Crop         4513 non-null   object 
 8   Fertilizer   4513 non-null   object 
dtypes: float64(1), int64(5), object(3)
memory usage: 317.4+ KB


In [27]:
df.head()

,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer
0,Black,75,50,100,6.5,1000,20,Sugarcane,Urea
1,Black,80,50,100,6.5,1000,20,Sugarcane,Urea
2,Black,85,50,100,6.5,1000,20,Sugarcane,Urea
3,Black,90,50,100,6.5,1000,20,Sugarcane,Urea
4,Black,95,50,100,6.5,1000,20,Sugarcane,Urea


# Step 3: EDA - Missing Values Analysis 

## Step 3)i): EDA - Show Missing Values in each Column

In [28]:
# Get percentage of null values in each column
null_values_percentage = df.isnull().mean().round(4).mul(100).sort_values(ascending=False)
print('-' * 44)
print("Percentage(%) of null values in each column")
print('-' * 44)
print(null_values_percentage)
print('\n')

# Get total null values in each column
total_null_values = df.isnull().sum().sort_values(ascending=False)
print('-' * 33)
print("Total null values in each column")
print('-' * 33)
print(total_null_values)

--------------------------------------------
Percentage(%) of null values in each column
--------------------------------------------
Soil_color     0.0
Nitrogen       0.0
Phosphorus     0.0
Potassium      0.0
pH             0.0
Rainfall       0.0
Temperature    0.0
Crop           0.0
Fertilizer     0.0
dtype: float64


---------------------------------
Total null values in each column
---------------------------------
Soil_color     0
Nitrogen       0
Phosphorus     0
Potassium      0
pH             0
Rainfall       0
Temperature    0
Crop           0
Fertilizer     0
dtype: int64


# Step 4: EDA - Duplicate Values Analysis 

## Step 4)i): EDA - Show Duplicate Values Rows

In [29]:
# Get percentage of duplicate rows
total_rows = len(df)
duplicate_rows = df.duplicated().sum()
duplicate_percentage = (duplicate_rows / total_rows) * 100

print('-' * 48)
print("Percentage(%) of duplicate rows in the DataFrame")
print('-' * 48)
print(f"{duplicate_percentage:.2f}%")
print('\n')

# Get total number of duplicate rows
print('-' * 30)
print("Total number of duplicate rows")
print('-' * 30)
print(duplicate_rows)


------------------------------------------------
Percentage(%) of duplicate rows in the DataFrame
------------------------------------------------
0.00%


------------------------------
Total number of duplicate rows
------------------------------
0


# Step 5: EDA - Analyzing Column

## Step 5)i): EDA - Univariate Analysis

In [30]:
def univariate_analysis_plotly(df):
    """

    Perform univariate analysis on a DataFrame using Plotly.

    Parameters:
    - df: DataFrame to be analyzed.

    Returns:
    - Interactive Plotly plots with summary statistics in the legend.
    """
    colors = px.colors.qualitative.Plotly

    for idx, column in enumerate(df.columns):

        # Generate descriptive statistics
        stats = df[column].describe()
        stats_str = '<br>'.join([f'{k}: {v:.2f}' if isinstance(v, (float, int)) else f'{k}: {v}' for k, v in stats.items()])

        # Visualization based on datatype
        if np.issubdtype(df[column].dtype, np.number):

            # If the column is numeric, plot a histogram with a box plot as marginal
            fig = px.histogram(df, x=column, marginal="box", title=f"Histogram for {column}", color_discrete_sequence=[colors[idx % len(colors)]])
        else:

            # If the column is categorical or textual, plot a bar chart
            value_counts = df[column].value_counts()

            fig = px.bar(x=value_counts.index, y=value_counts.values, 
                         title=f"Bar Chart for {column}", 
                         labels={"x": column, "y": "Count"},
                         color_discrete_sequence=[colors[idx % len(colors)]])

        # Add descriptive stats as a legend using a dummy trace for both types of columns
        fig.add_trace(go.Scatter(x=[None], y=[None], mode="lines", 
                                 name=stats_str, showlegend=True, 
                                 hoverinfo="none", opacity=0))
        
        fig.show()

univariate_analysis_plotly(df)

# Step 6): EDA - Feature Selection

## Step 6)i): EDA - Heatmap

In [31]:

def apply_auto_ordinal_encoding(df: pd.DataFrame, columns_to_encode: list[str]) -> pd.DataFrame:
    """
    Apply automatic Ordinal Encoding to specific columns of a DataFrame.

    Parameters:
    - df: Input DataFrame
    - columns_to_encode: List of column names to apply Ordinal Encoding

    Returns:
    - DataFrame with Ordinally Encoded columns
    """
    
    df_encoded = df.copy()
    
    for column in columns_to_encode:
        unique_values = df[column].unique()
        ordinal_mapping = {key: val for val, key in enumerate(unique_values)}
        
        # Print the ordinal mapping for the column
        print(f"Ordinal Encoding for '{column}': {ordinal_mapping}")
        
        df_encoded[column] = df[column].map(ordinal_mapping)
    
    return df_encoded


df_encoded_eda = apply_auto_ordinal_encoding(df, ['Soil_color', 'Crop', 'Fertilizer', ])
df_encoded_eda.head()

Ordinal Encoding for 'Soil_color': {'Black': 0, 'Red ': 1, 'Medium Brown': 2, 'Dark Brown': 3, 'Red': 4, 'Light Brown': 5, 'Reddish Brown': 6}
Ordinal Encoding for 'Crop': {'Sugarcane': 0, 'Jowar': 1, 'Cotton': 2, 'Rice': 3, 'Wheat': 4, 'Groundnut': 5, 'Maize': 6, 'Tur': 7, 'Urad': 8, 'Moong': 9, 'Gram': 10, 'Masoor': 11, 'Soybean': 12, 'Ginger': 13, 'Turmeric': 14, 'Grapes': 15}
Ordinal Encoding for 'Fertilizer': {'Urea': 0, 'DAP': 1, 'MOP': 2, '10:26:26 NPK': 3, 'SSP': 4, 'Magnesium Sulphate': 5, '13:32:26 NPK': 6, '12:32:16 NPK': 7, '50:26:26 NPK': 8, '19:19:19 NPK': 9, 'Chilated Micronutrient': 10, '18:46:00 NPK': 11, 'Sulphur': 12, '20:20:20 NPK': 13, 'Ammonium Sulphate': 14, 'Ferrous Sulphate': 15, 'White Potash': 16, '10:10:10 NPK': 17, 'Hydrated Lime': 18}


,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer
0,0,75,50,100,6.5,1000,20,0,0
1,0,80,50,100,6.5,1000,20,0,0
2,0,85,50,100,6.5,1000,20,0,0
3,0,90,50,100,6.5,1000,20,0,0
4,0,95,50,100,6.5,1000,20,0,0


In [32]:
import pandas as pd
import plotly.figure_factory as ff


def heatmap_correlations(df: pd.DataFrame, targetVariable:str ,colorscale:str="Viridis"):
    """
    Create a heatmap showing the correlation of all pairs of variables in the dataframe.
    Parameters:
    - df (pd.DataFrame): Data to be plotted.
    - targetVariable (str): The dependent variable for which correlations will be displayed.
    - colorscale (str): Desired colorscale for the heatmap. Default is "Viridis".
    Returns:
    - None: Shows the heatmap.
    """
    
    # Filtering only numerical columns
    df_numeric = df.select_dtypes(include=['number'])
    
    # Check if targetVariable is numeric
    if targetVariable not in df_numeric.columns:
        print(f"The target variable {targetVariable} is not numeric.")
        return
    
    # Print the correlation of the target variable with other variables
    print('-' * 52)
    print(f"Correlation of {targetVariable} with other Independent variables")
    print('-' * 52)
    print(df_numeric.corr()[targetVariable].sort_values(ascending=False))
    
    # Calculate the correlation matrix
    corr_matrix = df_numeric.corr()
    # Create a heatmap using the correlation matrix
    fig = ff.create_annotated_heatmap(
        z=corr_matrix.values, 
        x=list(corr_matrix.columns), 
        y=list(corr_matrix.index),
        annotation_text=corr_matrix.round(2).values,
        colorscale=colorscale
    )

    fig.update_layout(title="Correlation Heatmap of Variables")
    
    fig.show()




# heatmap_correlations(df, targetVariable="Class", colorscale='RdYlGn')
heatmap_correlations(df_encoded_eda, targetVariable="Fertilizer", colorscale='RdYlGn')


----------------------------------------------------
Correlation of Fertilizer with other Independent variables
----------------------------------------------------
Fertilizer     1.000000
Crop           0.459263
Soil_color     0.125628
Potassium     -0.044753
pH            -0.051754
Rainfall      -0.103483
Temperature   -0.124724
Phosphorus    -0.174454
Nitrogen      -0.214226
Name: Fertilizer, dtype: float64


In [33]:
"""
----------------------------------------------------
Correlation of Fertilizer with other Independent variables
----------------------------------------------------
Fertilizer     1.000000
Crop           0.459263
Soil_color     0.125628
Potassium     -0.044753
pH            -0.051754
Rainfall      -0.103483
Temperature   -0.124724
Phosphorus    -0.174454
Nitrogen      -0.214226
Name: Fertilizer, dtype: float64

"""


# # Drop All Columns with Less 10% Positive and Negative Correlation
# df.drop(["Potassium",
#          "pH"
#         ], axis=1, inplace=True)

# df.head()

'\n----------------------------------------------------\nCorrelation of Fertilizer with other Independent variables\n----------------------------------------------------\nFertilizer     1.000000\nCrop           0.459263\nSoil_color     0.125628\nPotassium     -0.044753\npH            -0.051754\nRainfall      -0.103483\nTemperature   -0.124724\nPhosphorus    -0.174454\nNitrogen      -0.214226\nName: Fertilizer, dtype: float64\n\n'

In [34]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# compute the vif for all given features
def compute_vif(dataframe: pd.DataFrame, numerical_columns:list, sort_ascending:bool=True):
    """
    Calculate the Variance Inflation Factor (VIF) for each feature in a DataFrame.
    
    Parameters:
    - dataframe (pd.DataFrame): The DataFrame containing the features.
    - numerical_columns (list): The list of numerical columns to calculate VIF for.
    - sort_ascending (bool): Whether to sort the VIF scores in ascending order. Default is True.
    """
    
    X = dataframe[numerical_columns]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    
    # Sort the VIF data
    vif.sort_values(by="VIF", ascending=sort_ascending, inplace=True)
    
    return vif


# Get Numerical Features
numerical_features = df.select_dtypes(include=np.number).columns.tolist()  # Get all numeric columns


# compute vif 
compute_vif(
    dataframe=df,
    numerical_columns=numerical_features,
    sort_ascending=False)


,Variable,VIF
1,Phosphorus,2.285658
0,Nitrogen,2.230467
2,Potassium,1.934949
4,Rainfall,1.407999
5,Temperature,1.132708
3,pH,1.080490


# Step 7): EDA - Feature Engineering/Scaling 

## Step 7)i): EDA - Categorical Feature Engineering/Scaling

In [35]:
df.head()

,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer
0,Black,75,50,100,6.5,1000,20,Sugarcane,Urea
1,Black,80,50,100,6.5,1000,20,Sugarcane,Urea
2,Black,85,50,100,6.5,1000,20,Sugarcane,Urea
3,Black,90,50,100,6.5,1000,20,Sugarcane,Urea
4,Black,95,50,100,6.5,1000,20,Sugarcane,Urea


In [36]:
import pandas as pd
import joblib

def apply_auto_ordinal_encoding(df: pd.DataFrame, columns_to_encode: list[str], save_path: str) -> pd.DataFrame:
    """
    Apply automatic Ordinal Encoding to specific columns of a DataFrame and save the encoding mapping.

    Parameters:
    - df: Input DataFrame
    - columns_to_encode: List of column names to apply Ordinal Encoding
    - save_path: Path to save the encoding mapping

    Returns:
    - DataFrame with Ordinally Encoded columns
    """
    
    df_encoded = df.copy()
    ordinal_mappings = {}
    
    for column in columns_to_encode:
        unique_values = df[column].unique()
        ordinal_mapping = {key: val for val, key in enumerate(unique_values)}
        
        # Print the ordinal mapping for the column
        print(f"Ordinal Encoding for '{column}': {ordinal_mapping}")
        
        df_encoded[column] = df[column].map(ordinal_mapping)
        ordinal_mappings[column] = ordinal_mapping
    
    # Save the ordinal mappings to a file
    joblib.dump(ordinal_mappings, save_path)
    
    return df_encoded


df_encoded = apply_auto_ordinal_encoding(df, 
                                         ['Soil_color', 'Crop', 'Fertilizer'], 
                                         'Mappings/ordinal_mappings.joblib')
df_encoded.head()


Ordinal Encoding for 'Soil_color': {'Black': 0, 'Red ': 1, 'Medium Brown': 2, 'Dark Brown': 3, 'Red': 4, 'Light Brown': 5, 'Reddish Brown': 6}
Ordinal Encoding for 'Crop': {'Sugarcane': 0, 'Jowar': 1, 'Cotton': 2, 'Rice': 3, 'Wheat': 4, 'Groundnut': 5, 'Maize': 6, 'Tur': 7, 'Urad': 8, 'Moong': 9, 'Gram': 10, 'Masoor': 11, 'Soybean': 12, 'Ginger': 13, 'Turmeric': 14, 'Grapes': 15}
Ordinal Encoding for 'Fertilizer': {'Urea': 0, 'DAP': 1, 'MOP': 2, '10:26:26 NPK': 3, 'SSP': 4, 'Magnesium Sulphate': 5, '13:32:26 NPK': 6, '12:32:16 NPK': 7, '50:26:26 NPK': 8, '19:19:19 NPK': 9, 'Chilated Micronutrient': 10, '18:46:00 NPK': 11, 'Sulphur': 12, '20:20:20 NPK': 13, 'Ammonium Sulphate': 14, 'Ferrous Sulphate': 15, 'White Potash': 16, '10:10:10 NPK': 17, 'Hydrated Lime': 18}


,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer
0,0,75,50,100,6.5,1000,20,0,0
1,0,80,50,100,6.5,1000,20,0,0
2,0,85,50,100,6.5,1000,20,0,0
3,0,90,50,100,6.5,1000,20,0,0
4,0,95,50,100,6.5,1000,20,0,0


In [37]:
df_encoded['Fertilizer'].value_counts().sort_index()

Fertilizer
0     1364
1      667
2      571
3      156
4      417
5      215
6       66
7      106
8      124
9      480
10     108
11       6
12       6
13      15
14      50
15      68
16      19
17      50
18      25
Name: count, dtype: int64

## Step 7)ii): EDA - Numerical Feature Engineering/Scaling

In [38]:
df_encoded.head()

,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer
0,0,75,50,100,6.5,1000,20,0,0
1,0,80,50,100,6.5,1000,20,0,0
2,0,85,50,100,6.5,1000,20,0,0
3,0,90,50,100,6.5,1000,20,0,0
4,0,95,50,100,6.5,1000,20,0,0


In [39]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scale_dataframe(dataframe, columns_to_scale):
    """
    Scales the specified columns of the DataFrame using Min-Max Scaling.
    :param dataframe: pandas DataFrame
    :param columns_to_scale: list of strings, names of columns to scale
    :return: DataFrame with scaled columns
    """
    # Create a copy of the DataFrame to avoid modifying the original one
    df_scaled = dataframe.copy()
    
    # Initialize the Min-Max Scaler
    scaler = MinMaxScaler()
    
    # Perform Min-Max Scaling on specified columns and update the DataFrame
    df_scaled[columns_to_scale] = scaler.fit_transform(dataframe[columns_to_scale])
    
    return df_scaled


numerical_features = [
    "Nitrogen",
    "Phosphorus",
    "Potassium",
    "pH",
    "Rainfall",
    "Temperature",
]

scaled_df = min_max_scale_dataframe(dataframe=df_encoded, 
                                    columns_to_scale=numerical_features)


df = scaled_df
df.head()

,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer
0,0,0.423077,0.5,0.655172,0.333333,0.5,0.333333,0,0
1,0,0.461538,0.5,0.655172,0.333333,0.5,0.333333,0,0
2,0,0.500000,0.5,0.655172,0.333333,0.5,0.333333,0,0
3,0,0.538462,0.5,0.655172,0.333333,0.5,0.333333,0,0
4,0,0.576923,0.5,0.655172,0.333333,0.5,0.333333,0,0


# Step 8) Train Test Split

In [40]:
df['Fertilizer'].value_counts().sort_index()  

Fertilizer
0     1364
1      667
2      571
3      156
4      417
5      215
6       66
7      106
8      124
9      480
10     108
11       6
12       6
13      15
14      50
15      68
16      19
17      50
18      25
Name: count, dtype: int64

### Train Test Split - SMOTENC

In [41]:
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split


X = df.drop("Fertilizer", axis=1)
y = df["Fertilizer"]


categorical_features_indices = [5]

# Apply SMOTE-NC
smote_nc = SMOTENC(sampling_strategy='auto', 
                   random_state=42, 
                   k_neighbors=5, 
                   n_jobs=-1,
                   categorical_features=categorical_features_indices
                   )

X_resampled, y_resampled = smote_nc.fit_resample(X, y)

# Split the resampled data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [42]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Generate distinct colors for each class using Plotly Express color scales
def generate_colors(n):
    color_scale = px.colors.qualitative.Plotly
    return [color_scale[i % len(color_scale)] for i in range(n)]

# Plot the class distribution before and after resampling
fig = make_subplots(rows=1, cols=2, subplot_titles=("Original Class Distribution", "Resampled Class Distribution after SMOTE"))

# Original class distribution
original_counts = y.value_counts().sort_index()
fig.add_trace(go.Bar(x=original_counts.index, y=original_counts.values, marker_color=generate_colors(len(original_counts)), showlegend=False), row=1, col=1)

# Resampled class distribution
resampled_counts = y_resampled.value_counts().sort_index()
fig.add_trace(go.Bar(x=resampled_counts.index, y=resampled_counts.values, marker_color=generate_colors(len(resampled_counts)), showlegend=False), row=1, col=2)

# Update layout
fig.update_layout(title_text="Class Distribution Before and After SMOTE", xaxis_title="Exited", yaxis_title="Count")

# Show plot
fig.show()

# Step 9) CatBoost Model

In [43]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def Confusion_Matrix_For_Multi_Class_With_Overview(title, y_test, y_pred):
    """
    Create a confusion matrix for multi-class classification with detailed overview.

    Parameters:
    - title: Title for the confusion matrix plot.
    - y_test: True labels of the test data.
    - y_pred: Predicted labels of the test data.

    Returns:
    - A seaborn heatmap representing the confusion matrix.
    """

    # Creating the confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)

    # Determine class labels
    class_labels = np.unique(np.concatenate((y_test, y_pred)))

    # Calculate the counts and percentages for the confusion matrix
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    
    # Calculate TP and FP percentages
    TP_percentages = ["{0:.2%}".format(value/np.sum(cf_matrix, axis=1)[i]) for i, value in enumerate(np.diag(cf_matrix))]
    FP_percentages = ["{0:.2%}".format((np.sum(cf_matrix, axis=0)[i] - value)/np.sum(cf_matrix)) for i, value in enumerate(np.diag(cf_matrix))]
    
    # Combine TP and FP with their percentages
    combined_info = []
    for i in range(cf_matrix.shape[0]):
        for j in range(cf_matrix.shape[1]):
            value = cf_matrix[i, j]
            if i == j:  # True Positive
                combined_info.append(f"{value}\n(TP: {TP_percentages[i]})")
            else:  # False Positive
                combined_info.append(f"{value}\n(FP: {FP_percentages[j]})")

    labels = np.asarray(combined_info).reshape(cf_matrix.shape)

    # Plotting the heatmap
    plt.figure(figsize=(25, 25))
    ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    ax.set_title(f'{title}\n\n')
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values')

    # Show the plot
    plt.show()


In [44]:
from catboost import CatBoostClassifier
import pandas as pd
from sklearn.metrics import classification_report

def CatBoost_Train_Evaluate(X_train, 
                            X_test,
                            y_train, 
                            y_test,
                            task_type=None,
                            iterations=2000, 
                            learning_rate=0.1,
                            loss_function='MultiClass', 
                            depth=6, 
                            l2_leaf_reg=3, 
                            border_count=32, 
                            verbose=100, 
                            random_seed=42):
    
    model = CatBoostClassifier(
        iterations=iterations,
        task_type=task_type,
        learning_rate=learning_rate,
        loss_function=loss_function,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        border_count=border_count,
        verbose=verbose,
        random_seed=random_seed,
        eval_metric='Accuracy'
    )

    model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)
    y_pred = model.predict(X_test).flatten()  # Ensure y_pred is a 1D array
    
    
    df = pd.DataFrame({'Actual': y_test.tolist(), 'Predicted': y_pred.tolist()})
    
    Confusion_Matrix_For_Multi_Class_With_Overview("CatBoost Confusion Matrix", y_test, y_pred)
    print(classification_report(y_test, y_pred))
    
    return df


pp = CatBoost_Train_Evaluate(X_train, X_test, y_train, y_test, task_type="GPU")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject